In [14]:
import requests
import time
import csv

API_KEY = '9e9ab06256174d82a01dc73f02418ca2'  # Reemplazar
BASE_URL = 'https://api.rawg.io/api/games'

def obtener_juegos_desde_1995(paginas_por_año=3):
    juegos_por_año = []

    for año in range(1995, 2025):  # de 1995 a 2024 inclusive
        juegos_este_año = []
        for pagina in range(1, paginas_por_año + 1):
            params = {
                'key': API_KEY,
                'dates': f'{año}-01-01,{año}-12-31',
                'page': pagina,
                'page_size': 40,
                'ordering': '-rating'
            }
            response = requests.get(BASE_URL, params=params)
            if response.status_code != 200:
                print(f"Error {response.status_code} en el año {año}, página {pagina}")
                break

            data = response.json()
            for juego in data.get('results', []):
                rating = juego.get('rating')
                if rating and rating > 0 and juego.get('genres'):
                    juegos_este_año.append({
                        'nombre': juego.get('name'),
                        'fecha_lanzamiento': juego.get('released'),
                        'año': año,
                        'puntuacion_media': rating,
                        'generos': ', '.join([g['slug'] for g in juego['genres']])
                    })

            if not data.get('next'):
                break
            time.sleep(1)  # evitar bloqueo

        if not juegos_este_año:
            # Añadir entrada vacía si no hay datos para el año
            juegos_por_año.append({
                'nombre': 'Sin datos',
                'fecha_lanzamiento': None,
                'año': año,
                'puntuacion_media': None,
                'generos': 'sin_datos'
            })
        else:
            juegos_por_año.extend(juegos_este_año)

    return juegos_por_año

def guardar_csv(datos, archivo='videojuegos_1995_2024_completo.csv'):
    with open(archivo, 'w', newline='', encoding='utf-8') as f:
        campos = ['nombre', 'fecha_lanzamiento', 'año', 'puntuacion_media', 'generos']
        writer = csv.DictWriter(f, fieldnames=campos)
        writer.writeheader()
        writer.writerows(datos)

# Ejecutar
if __name__ == '__main__':
    juegos = obtener_juegos_desde_1995(paginas_por_año=3)
    guardar_csv(juegos)


In [13]:
df

nombre fecha_lanzamiento   año  \
0                             Ascendancy        1995-01-02  1995   
1     The Incredible Machine Version 3.0        1995-01-01  1995   
2                          Ultimate Doom        1995-12-31  1995   
3               Ultimate Mortal Kombat 3        1995-11-06  1995   
4                            Road Rash 3        1995-05-15  1995   
...                                  ...               ...   ...   
3452             Lollipop Chainsaw RePOP        2024-09-11  2024   
3453                 Nobody Wants to Die        2024-07-16  2024   
3454         Please, Touch The Artwork 2        2024-02-19  2024   
3455                           Conscript        2024-07-23  2024   
3456                         Arctic Eggs        2024-05-16  2024   

      puntuacion_media                            generos genero_principal  
0                 4.50                           strategy         strategy  
1                 4.50                             puzzle           puzzle  
2                 4.43                    shooter, action          shooter  
3                 4.43                           fighting         fighting  
4                 4.42                             racing           racing  
...                ...                                ...              ...  
3452              3.77                             action           action  
3453              3.76                          adventure        adventure  
3454              3.75           casual, indie, adventure           casual  
3455              3.75  indie, shooter, adventure, action            indie  
3456              3.75       indie, adventure, simulation            indie  

[3457 rows x 6 columns]

In [18]:
import pandas as pd
import plotly.express as px

# Cargar los datos
df = pd.read_csv('videojuegos_1995_2024_completo.csv')

# Filtrar juegos con datos válidos
df = df.dropna(subset=['puntuacion_media', 'año', 'generos'])
df['año'] = df['año'].astype(int)

# Tomar solo el primer género de la lista
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'sin_datos')

# Agrupar por año y género, y calcular media de puntuación
pivot = df.groupby(['genero_principal', 'año'])['puntuacion_media'].mean().reset_index()

# Convertir a formato matriz para heatmap
heatmap_data = pivot.pivot(index='genero_principal', columns='año', values='puntuacion_media')

# Crear heatmap
fig = px.imshow(
    heatmap_data,
    labels=dict(x="Año", y="Género", color="Puntuación media"),
    title="Mapa de calor de la puntuación media por género y año (1995–2024)",
    text_auto=".2f",
    aspect="auto",
    color_continuous_scale="Viridis"
)

fig.update_layout(template="plotly_dark")
fig.show()


In [17]:
df

nombre fecha_lanzamiento   año  \
0                             Ascendancy        1995-01-02  1995   
1     The Incredible Machine Version 3.0        1995-01-01  1995   
2                          Ultimate Doom        1995-12-31  1995   
3               Ultimate Mortal Kombat 3        1995-11-06  1995   
4                            Road Rash 3        1995-05-15  1995   
...                                  ...               ...   ...   
3452             Lollipop Chainsaw RePOP        2024-09-11  2024   
3453                 Nobody Wants to Die        2024-07-16  2024   
3454         Please, Touch The Artwork 2        2024-02-19  2024   
3455                           Conscript        2024-07-23  2024   
3456                         Arctic Eggs        2024-05-16  2024   

      puntuacion_media                            generos  
0                 4.50                           strategy  
1                 4.50                             puzzle  
2                 4.43                    shooter, action  
3                 4.43                           fighting  
4                 4.42                             racing  
...                ...                                ...  
3452              3.77                             action  
3453              3.76                          adventure  
3454              3.75           casual, indie, adventure  
3455              3.75  indie, shooter, adventure, action  
3456              3.75       indie, adventure, simulation  

[3457 rows x 5 columns]

In [16]:
import pandas as pd
import plotly.express as px

#  Cargar archivo CSV
df = pd.read_csv('videojuegos_1995_2024_completo.csv')

#  Filtrar datos válidos
df = df.dropna(subset=['puntuacion_media', 'año'])
df['año'] = df['año'].astype(int)

#  Agrupar por año y calcular desviación estándar de las puntuaciones
dispersión = df.groupby('año')['puntuacion_media'].agg(['std', 'count', 'mean']).reset_index()
dispersión.rename(columns={'std': 'desviacion_estandar', 'count': 'n_juegos', 'mean': 'media_puntuacion'}, inplace=True)

#  Visualización de la dispersión por año
fig = px.bar(
    dispersión,
    x='año',
    y='desviacion_estandar',
    hover_data=['n_juegos', 'media_puntuacion'],
    title='Años con mayor dispersión en las puntuaciones de videojuegos (1995–2024)',
    labels={'desviacion_estandar': 'Desviación estándar', 'año': 'Año'}
)

fig.update_layout(template='plotly_dark', xaxis=dict(dtick=1))
fig.show()


**🧠 ¿Qué mide esto?**

Alta dispersión → año con muchos juegos muy bien y muy mal valorados.

Baja dispersión → puntuaciones más consistentes ese año (positiva o mediocre).

 Géneros dominantes vs. Puntuación media

**🧠 ¿Qué te permite ver este gráfico?**


Eje X → Popularidad (número de juegos por género).

Eje Y → Calidad media (puntuación).

Tamaño del punto → También representa popularidad.

Color → Género principal.


- Un género con muchos juegos pero baja puntuación → podría estar saturado (ej. sports).

Un género poco común pero muy bien valorado → nicho exitoso (ej. visual novel).

In [15]:
import pandas as pd
import plotly.express as px

#  Cargar datos
df = pd.read_csv('videojuegos_1995_2024_completo.csv')

#  Filtrar datos válidos
df = df.dropna(subset=['puntuacion_media', 'generos'])
df['genero_principal'] = df['generos'].apply(lambda x: x.split(',')[0] if isinstance(x, str) else 'sin_datos')

#  Agrupar por género principal
agrupado = df.groupby('genero_principal').agg(
    cantidad_juegos=('nombre', 'count'),
    puntuacion_media=('puntuacion_media', 'mean')
).reset_index()

#  Opcional: filtrar géneros con pocos juegos
agrupado = agrupado[agrupado['cantidad_juegos'] >= 20]

#  Visualización: Bubble plot
fig = px.scatter(
    agrupado,
    x='cantidad_juegos',
    y='puntuacion_media',
    size='cantidad_juegos',
    color='genero_principal',
    hover_name='genero_principal',
    title=' Géneros dominantes vs. Puntuación media',
    labels={
        'cantidad_juegos': 'Número de juegos',
        'puntuacion_media': 'Puntuación media'
    }
)

fig.update_layout(template='plotly_dark')
fig.show()
